In [16]:
import json
import nltk
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import numpy as np
import torch
import scipy
#nltk.download('punkt')
#nltk.download('punkt_tab')


model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

def sentiment_analysis(text):
    tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
    
    with torch.no_grad():
        input_sequence = tokenizer(text, return_tensors="pt", **tokenizer_kwargs)
        logits = model(**input_sequence).logits
        scores = {
        k: v
        for k, v in zip(
            model.config.id2label.values(),
            scipy.special.softmax(logits.numpy().squeeze()),
        )
    }

    return scores


def divide_body(body_text, max_tokens=512):
    if body_text=="":
        blocks = []
    else:
        sentences = nltk.sent_tokenize(body_text)
        blocks = []
        current_block = []
        current_tokens = 0

        for sentence in sentences:
            tokens = tokenizer.encode(sentence)
            if current_tokens + len(tokens) <= max_tokens:
                current_block.append(sentence)
                current_tokens += len(tokens)
            else:
                blocks.append(' '.join(current_block))
                current_block = [sentence]
                current_tokens = len(tokens)
        
        if current_block:
            blocks.append(' '.join(current_block))
        
    return blocks

def analyze_sentiment(text):
    if text == "":
        key2 = "positive"
        val2 = 0
    else:
        result = sentiment_analysis(text)
        key = max(result, key=result.get)
        if key == "neutral":
            order = list(dict(sorted(result.items(), key=lambda item: item[1])).items())
            val2 = max(result.values())*order[1][1]
            key2 = "positive" if val2>0 else "negative"
        else:
            key2 = key
            val2 = max(result.values())
        
    sentiment = 1 if key2 == 'positive' else -1
    print(result)
    return sentiment * val2

def weighted_sentiment_of_body(body_text):
    blocks = divide_body(body_text)
    if len(blocks) == 0:
        result = 0
    else:
        sentiment_scores = [analyze_sentiment(block) for block in blocks]
        print(sentiment_scores)
        weights = [(len(blocks)-i +1)/len(blocks) if len(blocks) > 1 else 1 for i in range(1, len(blocks)+1)]
        sum_weights = sum(weights)
        norm_weights = [i/sum_weights for i in weights]
        print(norm_weights)
        values = [sentiment_scores[i]*norm_weights[i] for i in range(len(blocks))]
        print(values)
        result = sum(values)
    return result

def final_sentiment(body_text, headline_text):
    body_sentiment = weighted_sentiment_of_body(body_text)
    headline_sentiment = analyze_sentiment(headline_text)

    num_sentences = len(nltk.sent_tokenize(body_text))
    if num_sentences < 4:
        final_score = 0.6 * headline_sentiment + 0.4 * body_sentiment
    else:
        final_score = 0.4 * headline_sentiment + 0.6 * body_sentiment

    return final_score



{'positive': np.float32(0.0032319964), 'negative': np.float32(0.98962444), 'neutral': np.float32(0.0071435533)}
{'positive': np.float32(0.5917818), 'negative': np.float32(0.37824628), 'neutral': np.float32(0.029971898)}
[np.float32(-0.98962444), np.float32(0.5917818)]
[0.6666666666666666, 0.3333333333333333]
[np.float32(-0.6597496), np.float32(0.1972606)]
{'positive': np.float32(0.0024538953), 'negative': np.float32(0.992841), 'neutral': np.float32(0.00470508)}
Sentimiento final: -0.674629807472229


In [17]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

input_json = '''
{
    "headline": "Yale’s Stephen Roach: Markets are in danger of being ‘whipsawed’ by Middle East conflict, U.S. unemployment",
    "body2":"",
    "body": "01 October 2024, Israel, Tel Aviv: Missiles launched from Iran are seen in the sky over Tel Aviv. Ilia Yefimovich/dpa | Picture Alliance | Getty Images\\n\\nMarkets are in danger of being \\"whipsawed\\" by the combination of regional conflict in the Middle East and rising unemployment in the United States, says Stephen Roach, senior fellow at Yale Law Schools Paul Tsai China Center. The conflict in the Middle East escalated on Tuesday, with Iran launching a ballistic missile attack on Israel after its killing of Hezbollah leader Hassan Nasrallah and an Iranian commander in Lebanon. Most Asian markets fell on Wednesday, tracking losses on Wall Street overnight, as investors fretted over rising tensions in the Middle East. \\"The markets really will not know where to turn,\\" Roach said, adding that conflicts in the Middle East are adding to inflationary risks at a time when global central banks are starting to ease monetary policy. \\"We are likely to see significant increases in volatility and markets that really are whipped back and forth dramatically,\\" Roach told CNBCs \\"Squawk Box Asia\\" on Wednesday.\\n\\nOil market impact\\n\\nThe Israel Defense Forces said its troops had started launching new strikes against Hezbollah targets in Lebanon in response to Irans missile attack Tuesday night. It remains to be seen whether there will be lasting effects on inflation, said Stephen Stanley, chief economist at Santander, adding that the oil market will be \\"affected more significantly\\" if the tension escalates.\\n\\nwatch now\\n\\nIran is the third-largest producer among the Organization of the Petroleum Exporting Countries, pumping out nearly four million barrels of oil per day, according to the Energy Information Administration. Oil prices had jumped over 5% after the missile strike before tapering to a 2% climb.\\n\\nOutlook for interest rates\\n\\nThe Israeli response to Irans attacks \\"might throw the Feds 25-basis-point rate cut off track,\\" said Kelvin Tay, regional chief investment officer at UBS Global Wealth Management. The U.S. Federal Reserve projected cutting interest rates by another half point over the next two policy meetings this year, according to the central banks so-called dot plot from the September meeting. Traders are also looking to U.S. payroll data on Friday for more indications on the state of the economy after the U.S. Federal Reserves jumbo rate cut in September. A higher-than-expected unemployment rate could prompt the Fed to accelerate the easing cycle to achieve a soft landing. The unemployment rate in September is expected to come in at 4.2%, according to data of a Reuters poll on LSEG, unchanged from the August figure. The unemployment rate had jumped to near a three-year high of 4.3% in July, a dramatic rise from the five-decade low of 3.4% in April 2023.\\n\\nwatch now"
    }
'''

data = json.loads(input_json)
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
# el actual da -3.5/5 = -0.7

{'positive': np.float32(0.0021294716), 'negative': np.float32(0.9411713), 'neutral': np.float32(0.056699213)}
{'positive': np.float32(0.9078003), 'negative': np.float32(0.07507241), 'neutral': np.float32(0.017127313)}
[np.float32(-0.9411713), np.float32(0.9078003)]
[0.6666666666666666, 0.3333333333333333]
[np.float32(-0.62744755), np.float32(0.30260012)]
{'positive': np.float32(0.0014736139), 'negative': np.float32(0.9956542), 'neutral': np.float32(0.0028721418)}
Sentimiento final: -0.593170166015625


In [27]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "Santander UK profits plunge after £295m motor finance hit",
    "body":"For free real time breaking news alerts sent straight to your inbox sign up to our breaking news emails Sign up to our free breaking news emails Sign up to our free breaking news emails SIGN UP I would like to be emailed about offers, events and updates from The Independent. Read our privacy policy\n\nBanking giant Santander UK has revealed profits plummeted in its third quarter as it took a £295 million hit following a major court decision on car finance commission.\n\nThe high street lender reported pre-tax profits of £143 million for the three months to the end of September, down 65% on the £413 million notched in the previous three months.\n\nThe group said it put aside £295 million as a provision to cover potential payouts as well as legal costs.\n\nBut it said there are “significant uncertainties as to the nature, extent and timing of any remediation action if required and the ultimate financial impact could be materially higher or lower than the amount provided”.\n\nSantander UK's pre-tax profits for the three months to the end of September\" data-source=\"\">\n\nThe group last month delayed its third quarter results after a Court of Appeal ruling saw the court side with consumers in a row over commission earned by companies selling car finance loans.\n\nThe news came as its Spanish owner Banco Santander also announced it was cutting more than 1,400 jobs across its UK business this year amid ongoing efforts to reduce costs.\n\nSantander UK said last month it disagrees with the conclusions reached by the Court of Appeal in its October ruling.\n\nIn the case, three people claimed they did not know their dealer was receiving more commission as a result of fixing a higher interest rate on their credit agreement.\n\nThe judgment therefore sets a precedent for the wider motor finance industry by ruling that any dealers receiving commission from lenders must ensure their customers are fully informed about the arrangement.\n\nSantander said the ruling “set a higher bar for the disclosure of and consent to the existence, nature, and amount of commission paid to dealers than that required by current FCA rules, or regulatory requirements in force at the time of the cases in question”.\n\n“The lenders involved in the cases subject to the Court of Appeal’s judgment have indicated that they intend to seek permission to appeal that judgment to the Supreme Court,” it added.\n\nThe group also revealed in its third quarter figures that mortgage lending had slumped by £5.5 billion since December last year, while customer deposits have reduced by £7.9 billion in the first nine months of 2024 after it cut rates on savings accounts to make them less attractive after forking out more to savers earlier in the year.\n\nSantander said it would “continue to prioritise profitability and our core banking franchise… resulting in lower mortgage lending and customer deposits in 2024”."
}
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el nuestro da 0

{'positive': np.float32(0.0019969596), 'negative': np.float32(0.99462456), 'neutral': np.float32(0.0033784336)}
{'positive': np.float32(0.0025385052), 'negative': np.float32(0.99282867), 'neutral': np.float32(0.0046328455)}
[np.float32(-0.99462456), np.float32(-0.99282867)]
[0.6666666666666666, 0.3333333333333333]
[np.float32(-0.6630831), np.float32(-0.3309429)]
{'positive': np.float32(0.005334133), 'negative': np.float32(0.99276006), 'neutral': np.float32(0.0019058301)}
Sentimiento final: -0.9935196042060852


In [29]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "Santander sets aside £295m for potential motor finance compensation",
    "body": "Santander UK's profits fell sharply in the third quarter as the lender set aside £295million for potential compensation related to motor finance commissions.\n\nThe bank last month delayed the publication of results after a Court of Appeal ruling saw the court side with consumers in a row over commission earned by companies selling car finance loans.\n\nSantander said it put aside £295million as a provision to cover potential payouts as well as legal costs.\n\nIt said there were 'significant uncertainties as to the nature, extent and timing of any remediation action if required and the ultimate financial impact could be materially higher or lower than the amount provided'.\n\nThe lender reported pre-tax profits of £143million for the three months to the end of September as a result, down nearly 75 per cent on the £558million notched up a year earlier.\n\nThe update came as its Spanish owner Banco Santander also announced it was cutting over 1,400 jobs across its UK business this year in a bid to reduce costs.\n\nMotor finance row hit profits at Santander UK\n\nLast month, Santander UK said it disagreed with the conclusions reached by the Court of Appeal in its October ruling.\n\nThe case saw three people claim they did not know their dealer was receiving more commission as a result of fixing a higher interest rate on their credit agreement.\n\nThe judgment sets a precedent for the wider motor finance industry by ruling that any dealers receiving commission from lenders must ensure their customers are fully informed about the arrangement.\n\nSantander said the ruling 'set a higher bar for the disclosure of and consent to the existence, nature, and amount of commission paid to dealers than that required by current FCA rules, or regulatory requirements in force at the time of the cases in question'.\n\nIt added: 'The lenders involved in the cases subject to the Court of Appeal’s judgment have indicated that they intend to seek permission to appeal that judgment to the Supreme Court.'\n\nUK lenders are set to face a £30billion compensation hit to settle the motor finance scandal.\n\nSantander also told shareholders on Wednesday that mortgage lending had fallen by £5.5billion since December last year, while customer deposits have reduced by £7.9billion in the first nine months of 2024 amid rate cuts on savings accounts.\n\nSantander said it would 'continue to prioritise profitability and our core banking franchise… resulting in lower mortgage lending and customer deposits in 2024'."
}
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el actual le da -3.5/5 = -0.7

{'positive': np.float32(0.0025506073), 'negative': np.float32(0.9936069), 'neutral': np.float32(0.0038424036)}
{'positive': np.float32(0.022211418), 'negative': np.float32(0.822274), 'neutral': np.float32(0.15551451)}
[np.float32(-0.9936069), np.float32(-0.822274)]
[0.6666666666666666, 0.3333333333333333]
[np.float32(-0.66240466), np.float32(-0.27409136)]
{'positive': np.float32(0.1991872), 'negative': np.float32(0.0072302413), 'neutral': np.float32(0.79358256)}
Sentimiento final: -0.4986690282821655


In [30]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "Rules will need to change if we want more cross-border market consolidation in Europe, Santander executive chair says",
    "body": "Share Share Article via Facebook Share Article via Twitter Share Article via LinkedIn Share Article via Email\n\nRules will need to change if we want more cross-border market consolidation in Europe, Santander executive chair says\n\nAna Botín, the executive chair of Santander, discusses the Spanish economy, European banking consolidation and the U.S. election at the Santander International Banking Conference in Madrid."
        }
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el actual le da 3.5/5 = 0.7

{'positive': np.float32(0.003376766), 'negative': np.float32(0.0022354135), 'neutral': np.float32(0.9943878)}
[np.float32(0.003357815)]
[1.0]
[np.float32(0.003357815)]
{'positive': np.float32(0.0041675926), 'negative': np.float32(0.021477334), 'neutral': np.float32(0.9743551)}
Sentimiento final: 0.013899057172238827


In [31]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "Martin Lewis explains 'good way' to make up to £200 for free before Christmas",
    "body": "Martin Lewis has explained how you could make up to £200 before Christmas by switching bank account. The MoneySavingExpert flagged five bank accounts that will pay you for your service - four of which will send you the cash before December 25. All of the bank switches flagged by Martin Lewis must be completed using the free Current Account Switch Service. But make sure you read all the terms and conditions first - as sometimes you may need to pay in a certain amount, or carry over direct debits.\n\nSpeaking during his Martin Lewis Money Show Live Christmas special, Martin said: “If you love your bank… then stick with it. I wouldn't tell you to leave. If not, then so many banks are willing to pay you.” He added: “It's a good way to make free cash and get it in time for Christmas.”\n\nWhen you switch bank accounts, your direct debits and standing orders will automatically be moved across, but for any recurring payments that use your long card number, most commonly with subscription services, you’ll need to update these manually. If you’re considering multiple switches, and you’ve got an important credit application coming up, Martin issued a word of warning. He said: “If you do one switch, well, it's an application on your credit file which has a very minor, short term negative effect. Lots of those in a short space of time can be an impact. Banks also like to see evidence of longevity, so that's also slightly negative. So I wouldn't switch my bank account if I had a mortgage application during the next two months.\n\n“Longer than that, I wouldn't care that much. It's only very minor. It doesn't really have a hit. Switching very regularly to make money can have a bigger impact. Do you have a use for your credit working? Do you have a debt application you want to make, or a mortgage application you want to make? No, well, in which case it'll have an effect, but why’d you care?”\n\nHow to make up to £200 switching bank accounts\n\nLloyds Bank: You'll get £200 if you switch to a Club Lloyds account by December 10. Club Lloyds customers can also choose from a range of lifestyle benefits, including a 12-month Disney+ subscription, a choice of six cinema tickets, magazine subscription, or Coffee Club and Gourmet Society membership.\n\nYou also get access to a linked regular saver paying 6.25% fixed for one year. To get the £200 switch cash, you need to move across three direct debits. Once you've done this, the £200 is paid within ten working days. You won't be eligible for the £200 switch cash if you've already received a switch bonus from Lloyds, Bank of Scotland or Halifax since April 2020. Keep in mind there is a £3 monthly fee with the Club Lloyds Account but this is waived if you pay in £2,000 a month.\n\nNatWest: NatWest is giving away £180 to those who move to its NatWest Reward account. You’ll need to pay in £1,250 into your new NatWest account and log into the mobile app within 60 days of switching accounts. Once you’ve done this, the £180 will be paid within seven days. The NatWest Reward account comes with a monthly fee of £2 - but it is possible to get £5 cashback, which wipes out this cost. Every month, you can get £4 cashback if you have two direct debits of £2 or more, plus £1 cashback by logging into the mobile app. NatWest also offers a linked Digital Regular Saver which pays 6% interest. You won't be eligible for the £180 switch award if you've previously received a switch bonus from NatWest, RBS or Ulster Bank before.\n\nFirst Direct: You can get £175 by switching to the First Direct 1st Account. For this switch bonus, you must pay in at least £1,000 and open two direct debits or standing orders from the account within 45 days of opening it. You must also register and log onto digital banking, within 45 days of the account opening and spend on your debit card five times.\n\nYou must be a \"new\" customer of First Direct to get this cash - this means you must not have had a bank account with them, or opened an HSBC current account on or after January 1, 2018. If you have a HSBC account, it must have been opened before this date. The £175 is paid by the 20th of the following month after you meet the switch bonus criteria. It's worth noting as well that you get access to a 7% regular saver for a year with the 1st Account.\n\nNationwide: Nationwide is giving £175 to those who switch their current account to its FlexDirect account. To qualify, you must transfer a minimum of two active direct debits, deposit a minimum of £1,000 and make one debit card payment within 31 days.\n\nYou also get access to a linked regular saver paying 6.5% variable interest for a year. You won't be eligible if you have switched into a Nationwide account, or have received switch cash from Nationwide, since August 18, 2021. The £175 is paid within 10 days of meeting all the criteria.\n\nSantander: You can get £150 from Santander - but be warned that the money won't arrive before Christmas. To get this cash, you need to switch across two direct debits and pay in at least £1,500. If you switch to the Santander Edge account, there is a £3 monthly fee - but you can get up to £20 a month cashback. This includes 1% on energy, council tax, broadband, TV, mobile and water bills, up to £10, as well another 1% on your spending on supermarket, fuel and trains, also up to £10. You also get a linked saver that pays 6% on up to £4,000 for the first year."
        }
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el actual le da 0

{'positive': np.float32(0.007742481), 'negative': np.float32(0.6531246), 'neutral': np.float32(0.33913293)}
{'positive': np.float32(0.009571464), 'negative': np.float32(0.03194253), 'neutral': np.float32(0.95848596)}
{'positive': np.float32(0.014526802), 'negative': np.float32(0.0069695804), 'neutral': np.float32(0.97850364)}
[np.float32(-0.6531246), np.float32(0.030616468), np.float32(0.014214529)]
[0.5000000000000001, 0.33333333333333337, 0.16666666666666669]
[np.float32(-0.3265623), np.float32(0.01020549), np.float32(0.0023690881)]
{'positive': np.float32(0.9891633), 'negative': np.float32(0.0029333786), 'neutral': np.float32(0.00790335)}
Sentimiento final: 0.20727266371250153


In [36]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "Should I fix my mortgage for two or five years right now?",
    "body":"My wife and I are remortgaging at the end of the year and trying to weigh up our options between a two-year and a five-year fixed rate mortgage.\n\nWhat's your view? There isn't a huge difference in monthly repayments between the two, but trying to get a sense of what will happen to interest rates in the future is a challenge.\n\nNew normal? Fixed mortgage rates have been heading higher in recent weeks, despite the Bank of England cutting interest rates earlier this month\n\nEd Magnus of This is Money replies: This question is being asked a lot at the moment. As households continue to roll off their fixed rate deals - some of whom may have been paying as little as 1 per cent interest, the stakes now seem much higher, given they now have to fix above 4 per cent. Although five-year fixes are usually slightly cheaper at the moment, the difference between two-year and five-year mortgage rates has narrowed in recent weeks making this decision even harder.\n\nWhat are today's mortgage rates?\n\nThe lowest two-year fixed remortgage rate is now 4.27 per cent compared to 4.07 per cent for the lowest five-year fix.\n\nOn a £200,000 mortgage being repaid over 25 years, that's the difference between paying £1,086 and £1,063 each month. The average five-year fix is now 5.22 per cent, according to Moneyfacts, compared to the average two-year rate of 5.5 per cent.\n\nHowever, two-year fixes are the product of choice at the moment.\n\nLast month, Santander said 60 per cent of customers were choosing two-year fixes at present. For many, this is because they hope interest rates will be lower when they come to remortgage in two years' time.\n\nLess than a quarter of Santander customers are opting for five-year fixed rate products, even though they are currently cheaper. The remainder are mostly choosing fixes lasting three or ten years, or trackers.\n\nWhen will mortgage rates fall?\n\nJust because the majority of households are fixing for two years, does not mean they are necessarily right.\n\nMany households will be opting for two-year fixes because the consensus is that interest rates will continue to fall and mortgage rates will follow suit. It is true that the Bank of England has started reducing the base rate - a trend which leads to lower mortgage rates in general.\n\nHowever, future interest rate cuts by the Bank of England are already baked into fixed rate mortgage pricing. This is why the lowest priced five-year fixed rate products are hovering just above 4 per cent, rather than above the Bank of England base rate at 4.75 per cent.\n\nPrior to the quickfire base rate rises between December 2021 and August 2023, the lowest mortgage rates have trended above base rate. That was the case at least between 2008 and 2022.\n\nMortgage pricing is largely based on Sonia swap rates - an inter-bank lending rate, based on future interest rate expectations.\n\nWhen Sonia swaps rise sufficiently it often results in fixed mortgage rates going up, and vice versa when they fall. As of 14 November, five-year swaps were at 4.01 per cent and two-year swaps were at 4.22 per cent. It is very rare for mortgage rates to go below the equivalent swaps.\n\nWhat happens to mortgage rate pricing in the future will depend largely on what happens to inflation as well as the wider economy. A sudden inflationary shock could send interest rates higher, while a recession could see the Bank of England cut rates more aggressively, for example.\n\nInflation forecast: The BoE expects the rate of inflation to hover just above 2% until 2027\n\nHow far will base rate fall?\n\nThe most bullish forecasters on rate cuts have base rate coming down to as low as 2.75 per cent by the end of 2025, with Goldman Sachs analysts announcing this rate forecast recently.\n\nGoldman's prognosis would mean a quarter point interest rate cut at all nine meetings of the Bank's Monetary Policy Committee (MPC) from December 2024 to November 2025. At the more reserved end of the spectrum, Santander recently revealed it expects interest rates to fall to 3.75 per cent by the end of next year.\n\nMeanwhile, economists at Capital Economics think the base rate will fall to 3.5 per cent by early 2026. They had previously forecast that interest rates would fall to 3 per cent by the end of next year, but have concluded that rates will now fall more slowly as a result of the Labour's first Budget.\n\nNew forecast: Capital Economics has changed its interest rate forecast because it now thinks the Bank of England will cut rates more slowly as a result of the budget\n\nLooking further ahead, it is anyone's guess. Santander predicts that base rate will remain between 3 per cent and 4 per cent for the foreseeable future.\n\nMeanwhile economists at Oxford Economics are forecasting they will fall to a low of 2 per cent in 2027.\n\nIf Santander's prediction plays out, a five-year fixed rate mortgage could well play out to be the better option. But of course, if Oxford Economics are right, then you may wish you had gone for the two-year fix.\n\nHow long should you fix your mortgage?\n\nSo with all that in mind, how long should mortgage holders fix for now?\n\nWe spoke to Chris Sykes, technical director of broker Private Finance and Mark Harris, chief executive of mortgage broker SPF Private Clients.\n\nChris Sykes replies: There are a few things to consider to help make your decision.\n\nFirst, the cost. Often mortgage deals have product fees attached to them, and there can also be legal fees, broker fees and early repayment charges to consider. It is worth taking these into consideration. Often people just look at the rate, but paying a £999 product fee every two years instead of every five years can add up over time, especially if added to the loan amount.\n\nWhat is loan-to-value?"
        }
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el actual le da -3.5/5 = -0.7

{'positive': np.float32(0.0120294755), 'negative': np.float32(0.6229346), 'neutral': np.float32(0.3650359)}
{'positive': np.float32(0.003992115), 'negative': np.float32(0.07372681), 'neutral': np.float32(0.922281)}
{'positive': np.float32(0.0039350647), 'negative': np.float32(0.035746727), 'neutral': np.float32(0.9603182)}
[np.float32(-0.6229346), np.float32(0.06799684), np.float32(0.034328233)]
[0.5000000000000001, 0.33333333333333337, 0.16666666666666669]
[np.float32(-0.3114673), np.float32(0.022665612), np.float32(0.0057213726)]
{'positive': np.float32(0.0064561875), 'negative': np.float32(0.5273671), 'neutral': np.float32(0.46617675)}
Sentimiento final: -0.38079503178596497


In [41]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "'Crazy growth': How one product created a multi-million dollar brand",
    "body":"'Crazy growth': How one product created a multi-million dollar brand\n\nNell Diamond, CEO of Hill House, shares how her small business skyrocketed to global success with a simple, singular product. At the height of the Covid-19 pandemic, Hill House Home's Nap Dress became more than just a piece of clothing – it was a symbol of comfort and versatility for a world in flux. What started as a direct-to-consumer bedding and home business in 2016 had grown into a fashion movement, reflecting how a single dress could adapt to your body over the years and transform depending on the demands of the day. The company introduced the Nap Dress in 2019, a design that leaned into the idea of smocked fabric from the 1950s and reimagined it with modern, universal appeal. It didn't take long for the dress – now with over 50 designs – to go viral on social media and become common in many closets around the world.\n\n\"Our crazy growth happened from 2019 to 2020 – right in the middle of quarantine and while I was pregnant with twins,\" CEO Nell Diamond tells the BBC. \"I could obviously see how much the business was changing internally from the sales volume, but one really pivotal moment for me was working from home, sitting in my bedroom in New York City, and looking out the window to see someone walking down the street wearing one of our dresses.\n\nEXECUTIVE LOUNGE The BBC's series features interviews with executive leaders making innovative, data-driven decisions helping shape the future of business – and paving the path for other leaders to thrive. Read more conversations here.\n\n\"Entrepreneurship can feel really lonely and insular, so to realise that people know about your little project is incredibly rewarding. I'll never forget that moment.\"\n\nSince then, the company has sold over one million Nap Dresses, expanded into categories like outerwear and swimwear, and opened five retail locations across the United States, from New York to Charleston. The brand's reach has also expanded within other retailers, including Shopbop and Saks. Hill House puts its current valuation at approximately $150m (£118m), although the BBC was unable to obtain an independent valuation. The company has faced challenges, too, including navigating global supply chain disruptions during the pandemic and scaling operations to meet increased demand. But at a time when small businesses struggle to stand out amid economic uncertainty, Hill House's story underscores the importance of adaptability and building strong connections with consumers.\n\n\"We've had the biggest year in company history,\" Diamond said. \"The business has continued to grow past even our optimistic plans.\"\n\nWhen we told our customers what was happening, people loved having that insight into the humanity behind the products they're actually buying\n\nDiamond's love of fashion started early, during her teenage years while attending the American School in London, UK. After graduating from Princeton University, she joined the trading desk of a finance firm, initially entering the same sector of business as her father Bob Diamond, the former CEO of Barclays bank. She quickly realised, however, that her passion lay elsewhere.\n\n\"I was always drawn to retail,\" she says. \"I'd steal my friends' equity research papers to learn what companies were doing in the retail space. I realised that this thing which started as a guilty pleasure – loving fashion – turned out to be a viable career opportunity.\"\n\nNow, as CEO, Diamond oversees every aspect of the business, from growth strategy to creative direction. Below, she talks with the BBC about the company's biggest challenges, its rapid evolution into a lifestyle brand and plans for the future.\n\nI wanted to bring a design-forward point of view to the home category. We started with just home products: bedding, pillows, a little bit of pyjamas and robes. But I really focused on the home and, in particular, the bedroom, drawing off of some of the design elements of my London upbringing – great British brands like [ interior decorating firm] Colfax and Fowler and amazing prints that I had seen in the Victoria and Albert Museum. There were many businesses launching direct-to-consumer [ approaches] and talking about cutting out the middleman. I wanted to do the same thing and … start small and focus on just one or two products. While I was at business school, I spent time on the little parts of starting a business – trademarks, legal documents, hiring my first few contractors and setting up our Instagram account. After graduating, I spent the first couple of years bootstrapping the business and really focusing on meeting our customers. It was a very small team in a co-working space in New York City.\n\nBy 2019, everything dramatically changed overnight. We went from low and slow and careful growth to the success of the Nap Dress. Within a few months, our business was majority a fashion business based off the strength of this one product line. We went from five people to 30 people and from one store to now having almost six stores. It really changed the scale of the business. I remember our very first order was for a hundred units and being petrified that we wouldn't actually sell them. To have sold a million of them now is really crazy.\n\nHill House Home The Nap Dress has proved wildly popular with consumers, with new collections often selling out in minutes of going on sale (Credit: Hill House Home)\n\nWhy do you think this one particular product, the Nap Dress, resonates with so many people?\n\nIt's so many different things. We didn't invent smocking, which is what makes our dresses most identifiable. My grandmother was wearing smock dresses in the 1950s. Juicy Couture [ the Los Angeles-based clothing brand] popularised smocking and terry cloth in the 2000s. But I think what was so important to us was figuring out a very proprietary type of smocking that could stretch with you. I wore it all throughout my twin pregnancy and then the snap back."
        }
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el actual le da -0.21/5 = -0.04

{'positive': np.float32(0.82555616), 'negative': np.float32(0.0028381064), 'neutral': np.float32(0.1716058)}
{'positive': np.float32(0.19116665), 'negative': np.float32(0.0050330237), 'neutral': np.float32(0.80380034)}
{'positive': np.float32(0.26198006), 'negative': np.float32(0.042209398), 'neutral': np.float32(0.69581056)}
[np.float32(0.82555616), np.float32(0.15365982), np.float32(0.18228848)]
[0.5000000000000001, 0.33333333333333337, 0.16666666666666669]
[np.float32(0.41277808), np.float32(0.05121994), np.float32(0.030381415)]
{'positive': np.float32(0.9859509), 'negative': np.float32(0.003955651), 'neutral': np.float32(0.010093536)}
Sentimiento final: 0.6910080313682556


In [43]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "Spanish banks first to onboard to EU instant cross-border payment scheme",
    "body":"The Spanish banking community has become the first EU market to enable instant cross-currency payments under the European Payment Council's OCT Inst (One-Leg Out (OLO) Instant Credit Transfer scheme.\n\n1 Like 5\n\nThis content has been selected, created and edited by the Finextra editorial team based upon its relevance and interest to our community.\n\nThe entire Spanish banking community has been fully onboarded following pilot trials conducted in June 2023 with CaixaBank, Santander, BBVA, Swift, Iberpay and several international banks in different countries.\n\n\n\nOCT Inst is a cross-currency payment scheme to support the processing of incoming and outgoing international instant account-to-account based credit transfers. It is the first EPC scheme which covers exclusively the Euro Leg of international instant credit transfers entering or leaving the geographical scope of Sepa.\n\n\n\nThis is designed to ensure faster execution of international payments, more transparency on costs and better payment status traceability.\n\n\n\nThe scheme went live in November 2023 but the postponement of ISO20022 migration for all Sepa schemes led to PSPs and Clearing and Settlement Mechanisms (CSMs) replanning the start of operations.\n\nSpeaking in May when Santander became the first Spanish bank to onboard to the scheme, Juan Luis Encinas, CEO, Iberpay, commented: \"Providing the infrastructure and services necessary for processing international cross-currency instant transfers is a significant step forward for Iberpay, enabling our banks to offer new innovative services, to improve the customer experience and to remain highly competitive in the global payments industry.\""
        }
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el actual le da 0

{'positive': np.float32(0.99498785), 'negative': np.float32(0.0016573659), 'neutral': np.float32(0.0033548172)}
[np.float32(0.99498785)]
[1.0]
[np.float32(0.99498785)]
{'positive': np.float32(0.9908488), 'negative': np.float32(0.0034096127), 'neutral': np.float32(0.005741586)}
Sentimiento final: 0.9933322668075562


In [46]:
model_path = 'v1final_model'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

data = {
    "headline": "The self-employed are natural entrepreneurs. So why is our government ignoring them?",
    "body":"The self-employed are natural entrepreneurs. So why is our government ignoring them?\n\nSupporting the self-employed could help unleash a needed wave of entrepreneurialism. So why is the government not acting? Camilla Esmund, senior manager at Interactive Investor, asks in today’s Notebook\n\nSupport needed for the self-employed\n\nOver 4m people in the UK have decided to be their own boss, fuelled by the entrepreneurial dream or a better work-life balance. And although being your own boss has obvious appeal, these freedoms come with real financial challenges that can be more difficult to navigate when you’re self-employed.\n\nThere are several measures we’d like to see put in place to support self-employed workers. After all, this is not just about fairness, it’s about recognising the vital role self-employed individuals play in our economy. It has been a big year for pensions in 2024, with rumoured pension reform dominating headlines, and the promise of a broader ‘pensions review’ from the new Labour government, which we are waiting to see. But, we are yet to see anything that speaks to the unique challenges faced by the self-employed. They seem to be ignored or forgotten. This neglect is worrying, given the increasing number of people opting for (or being pushed into) full-time self-employment. And at a time when we hear so much rhetoric about supporting and nourishing entrepreneurial spirit in the UK. We published data earlier this year which should ring alarm bells for policymakers. In a self-employed survey using Opinium Research, we found that 60 per cent have less than £10,000 in pension wealth, and 38 per cent have no pension savings at all. Once again, the gender pensions gap shows itself, too – with 43 per cent of women having no pension compared to 35 per cent of men. But this isn’t just about pensions, self-employed workers are also dangerously financially exposed more broadly. In fact, 15 per cent have no cash savings at all, and 31 per cent have less than £1,000 saved. Plus, our research also found that most self-employed workers do not have any other significant investment.\n\nIt’s high time policymakers paid greater attention to this often-overlooked segment of the workforce. There needs to be a concerted effort to integrate self-employed workers into the pension conversation, offering them a comprehensive level of support and resources in the same league as what’s available to salaried employees. At Interactive Investor, we have long been very vocal on the need for greater financial literacy in the UK, but let’s also see some enhanced pension education to actually improve the awareness of existing services like Pension Wise to help the self-employed make informed retirement decisions. We’d also like to see the expansion of the Pensions Wise Access service, offering the service to self-employed individuals of all ages, automatically scheduling appointments when they register as sole traders or submit tax returns. Think of the difference even those changes could make.\n\nWe’d also like to see the implementation of an automatic opt-in model linked to the self-assessment tax return to increase pension contributions. And finally, but crucially – pension dashboards. We’d like to see policymakers develop dashboards tailored to the self-employed which will help them get a clear view of their retirement savings and have better management of contributions. The reality of financial abuse in relationships\n\nThis week, I saw new research from Gina Miller’s organisation Moneyshe, stating that a third of women feel trapped in a relationship or situation due to a lack of financial independence. Their campaign is calling for women to invest to fight back against financial abuse, raising awareness of the gender investment gap in the UK and the importance of personal financial resilience and security.\n\nI completely agree that helping more women build their own financial independence and resilience is vital. At Interactive Investor, we’ve done a lot of work on the various ‘gaps’ when it comes to women and their money. But, we must remember that financial abuse, a form of coercive control, is complex. Sadly, a staggering one in six women in the UK has experienced financial abuse in a relationship, according to the charity Surviving Economic Abuse. And though still not talked about enough, some very courageous female commentators in my industry have spoken about financial abuse in depth, and have outlined steps women can take to get help. The Surviving Economic Abuse website has a range of useful resources, as well as helplines you can call if you or someone you know is in a financially abusive relationship. Just know that help is available.\n\nMaking your money work hard under inflation\n\nLast week, inflation nudged back above the Bank of England’s two per cent target in October. Hitting 2.3 per cent, it was a sharp increase of 0.6 per cent, and with both services and core inflation speeding up last month, too, the prospect of a December interest rate cut is now looking faint. This is a crucial reminder of why you need to ensure your savings and investments work harder to grow in real terms. This means checking how much you’re earning on your savings, and shopping around for the best deals.\n\nRemember, investing in the stock market gives you the potential to grow your money significantly faster than inflation and savings accounts, albeit with higher risk. So, it will always depend on your individual circumstances, but if you’re in it for the long term, your investments will have longer to recover if the market takes a dip. Plus, regular, monthly investments can also help smooth out the impact of stock market volatility on returns from investing.\n\nAnd, we can’t talk about investing without talking about keeping an eye on fees. The good news is investment costs are one of the few things you can control. Investing will never be free, but it doesn’t have to be expensive either! Keeping your fees low can help your portfolio outpace inflation by a wider margin."
        }
headline = data['headline']
body = data['body']

final_score = final_sentiment(body, headline)

print(f"Sentimiento final: {final_score}")
#el actual le da 0

{'positive': np.float32(0.0026992005), 'negative': np.float32(0.9917354), 'neutral': np.float32(0.005565392)}
{'positive': np.float32(0.51624167), 'negative': np.float32(0.078767136), 'neutral': np.float32(0.40499118)}
{'positive': np.float32(0.97952616), 'negative': np.float32(0.010770004), 'neutral': np.float32(0.00970377)}
[np.float32(-0.9917354), np.float32(0.51624167), np.float32(0.97952616)]
[0.5000000000000001, 0.33333333333333337, 0.16666666666666669]
[np.float32(-0.4958677), np.float32(0.17208056), np.float32(0.16325437)]
{'positive': np.float32(0.0018220713), 'negative': np.float32(0.98754114), 'neutral': np.float32(0.010636793)}
Sentimiento final: -0.4913361370563507
